# Chains in LangChain

### Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [11]:
import pandas as pd

df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]
llm_model = "gemma-3-27b-it" # "gemini-2.0-flash-lite"

llm = ChatGoogleGenerativeAI(
    model=llm_model,
    temperature=0.9,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give your answer in one short paragraph"
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "Queen Size Sheet Set"
chain.run(product)

C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_32744\1694080553.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_32744\1694080553.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Here are a few options, ranging in style, for a company selling Queen Size Sheet Sets:\n\n**Cozy Queen Linens** is a strong choice – it\'s descriptive, evokes comfort, and clearly states the focus. Other good options include **Queen Dream Sleep**, **The Queen\'s Bed**, or **Simply Queen Sheets** if you want something more straightforward. Ultimately, the "best" name depends on your brand\'s overall aesthetic (luxury, budget-friendly, modern, etc.), but prioritizing clarity and a feeling of comfort is key for bedding.'

In [ ]:
## LangChain 0.3

chain = prompt | llm

product = "Queen Size Sheet Set"

chain.invoke({"product": product})

AIMessage(content='Here are a few options, ranging in style, for a company selling Queen Size Sheet Sets:\n\n**Cozy Queen Linens** is a strong choice – it\'s descriptive, evokes comfort, and clearly states the focus. Other good options include **Queen Dream Sheets**, **The Queen\'s Bed**, or **Simply Queen Sheets**. If you want to emphasize quality, **Royal Queen Collection** or **Queen Comforts** could work. Ultimately, the "best" name depends on your brand\'s overall aesthetic and target audience, but prioritizing clarity and a feeling of comfort is key.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1baecfd9-13a3-4953-87fc-eebdd8322e1d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 119, 'total_tokens': 144, 'input_token_details': {'cache_read': 0}})

## Simple Sequential Chain

In [6]:
from langchain.chains import SimpleSequentialChain

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give your answer in two short paragraph"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are a few name ideas, broken down by approach, with my top picks at the end:

**Descriptive & Functional:** "Queen Comforts," "Queen Size Linens," "The Queen Sheet Co.," "Restful Queen," or "Simply Queen Sheets" clearly state what you sell. **Elegant & Evocative:** "Serene Slumber," "Royal Rest," "Dream Weaver Linens," or "The Velvet Queen" suggest a luxurious experience. **Modern & Minimalist:** "Q Sleep," "Loom Queen," or "The Sheet Edit" are short, memorable, and appeal to a contemporary aesthetic.

**My top recommendations are:** **"Queen Comforts"** – it's straightforward, inviting, and highlights the benefit. Or, **"Serene Slumber"** – if you're aiming for a more premium, relaxing brand image. Ultimately, the "best" name depends on your target audience and the overall brand you want to create. Check for trademark availability before settling on a name!
Here are 20-word descriptions for both "Queen Comforts" & "Serene Slumber," 

'Here are 20-word descriptions for both "Queen Comforts" & "Serene Slumber," reflecting the provided brand ideas:\n\n**Queen Comforts:** Luxurious, high-quality queen size sheets & bedding designed for ultimate comfort & restful sleep. Affordable indulgence for your nightly retreat.\n\n**Serene Slumber:** Experience blissful, restorative sleep with our elegantly crafted queen linens. Indulge in luxurious comfort & wake up feeling refreshed & renewed.'

In [ ]:
simple_chain = first_prompt | llm | (lambda response: {"company_name": response.content}) | second_prompt | llm

simple_chain.invoke({"product": product}).content

'Here are a few 20-word descriptions, tailored to the name options:\n\n**For "Queen Comforts":**\n\nLuxurious queen-size sheets & bedding designed for ultimate comfort. Experience restful nights and wake refreshed with Queen Comforts\' quality linens.\n\n**For "Serene Slumber":**\n\nIndulge in serene nights with Serene Slumber\'s exquisitely crafted queen bedding. Experience luxurious comfort and wake up feeling truly rejuvenated.'

## Sequential Chain

In [13]:
from langchain.chains import SequentialChain

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short and polite follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.',
 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit or old batch due to its inferiority to store-bought versions.",
 'followup_message': 'Here are a few options, ranging from slightly more formal to slightly more casual, all polite and appropriate:\n\n**Option 1 (More Formal):**\n\n> Bonjour,\n\n> Nous vous remercions pour votre ret

In [21]:
from langchain_core.output_parsers import StrOutputParser

# Create the chain using the pipe operator
translate_chain = first_prompt | llm | StrOutputParser()
summarize_chain = second_prompt | llm | StrOutputParser()
language_chain = third_prompt | llm | StrOutputParser()
followup_message = fourth_prompt | llm | StrOutputParser()

# Define the complete workflow
def process_review(review):
    # Run translation and store result
    english_review = translate_chain.invoke({"Review": review})
    
    # Run summarization using the translated review
    summary = summarize_chain.invoke({"English_Review": english_review})
    
    # Determine language
    language = language_chain.invoke({"Review": review})
    
    # Generate followup
    followup_result = followup_message.invoke({"summary": summary, "language": language})
    
    return {
        "English_Review": english_review,
        "summary": summary,
        "language": language,
        "followup_message": followup_result
    }

result = process_review(df.Review[5])
result

{'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.',
 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit or old batch due to its inferiority to store-bought versions.",
 'language': "The language is **French**.\n\nHere's a quick translation to confirm:\n\n* **Je trouve le goût médiocre.** - I find the taste mediocre.\n* **La mousse ne tient pas, c'est bizarre.** - The foam doesn't hold, it's strange.\n* **J'achète les mêmes dans le commerce et le goût est bien meilleur...** - I buy the same ones in stores and the taste is much better...\n* **Vieux lot ou cont

In [15]:
from langchain_core.runnables import RunnablePassthrough

# Create the chain using the pipe operator
translate_chain = first_prompt | llm | RunnablePassthrough()
summarize_chain = second_prompt | llm | RunnablePassthrough()
language_chain = third_prompt | llm | RunnablePassthrough()

# Define the complete workflow
def process_review(review):
    # Run translation and store result
    english_review = translate_chain.invoke({"Review": review})
    
    # Run summarization using the translated review
    summary = summarize_chain.invoke({"English_Review": english_review})
    
    # Determine language
    language = language_chain.invoke({"Review": review})
    
    # Generate followup
    followup_message = fourth_prompt | llm | RunnablePassthrough()
    followup_result = followup_message.invoke({"summary": summary, "language": language})
    
    return {
        "English_Review": english_review,
        "summary": summary,
        "language": language,
        "followup_message": followup_result
    }

result = process_review(df.Review[5])
result

{'English_Review': AIMessage(content='Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0a6ef3a8-54fb-45d9-81f9-83b7f509be69-0', usage_metadata={'input_tokens': 54, 'output_tokens': 94, 'total_tokens': 148, 'input_token_details': {'cache_read': 0}}),
 'summary': AIMessage(content='This reviewer is dissatisfied with the product, finding the taste subpar, the foam strange, and suspects it may be an old or counterfeit batch due to its inferior quality compared to store-bought versions.', add

In [20]:
# Create the chain using the pipe operator
translate_chain = first_prompt | llm | StrOutputParser()
summarize_chain = second_prompt | llm | StrOutputParser()
language_chain = third_prompt | llm | StrOutputParser()
followup_chain = fourth_prompt | llm | StrOutputParser()

# Define the complete workflow using `|`
pipeline = (
    RunnablePassthrough.assign(Review=lambda x: x["Review"])  # Extract Review
    | {
        "English_Review": translate_chain,  # Translate review to English
        "language": language_chain         # Detect language
    }
    | {
        "English_Review": RunnablePassthrough(),  # Ensure it’s passed forward
        "summary": summarize_chain,  # Summarize the English review
        "language": RunnablePassthrough()  # Pass language forward
    }
    | {
        "summary": RunnablePassthrough(),       # Summarize the translated review
        "language": RunnablePassthrough(),  # Pass language forward
        "followup_message": followup_chain  # Generate follow-up message
    }
)

# Run the pipeline
result = pipeline.invoke({"Review": df.Review[5]})

print(result)

{'summary': {'English_Review': {'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.', 'language': "The language is **French**.\n\nHere's a quick translation:\n\n* **Je trouve le goût médiocre.** - I find the taste mediocre.\n* **La mousse ne tient pas, c'est bizarre.** - The foam doesn't hold, it's strange.\n* **J'achète les mêmes dans le commerce et le goût est bien meilleur...** - I buy the same ones in stores and the taste is much better...\n* **Vieux lot ou contrefaçon !?** - Old batch or counterfeit!?"}, 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit o